In [12]:
import torch
import os.path as path
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
import pandas as pd
import tqdm
from torch_geometric.nn import HeteroConv, GATConv, SAGEConv, Linear
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import torch.nn.functional as F

In [2]:
data_folder = "ds/"
test_hd = "full_hd.pt"
train_collab_with_filename = "collab_with_2020_3.pt"
best_threshold = 0.73

In [3]:
data = torch.load(path.join(data_folder, test_hd))

data.validate()

/tmp/ipykernel_89638/624821894.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path.join(data_folder, test_hd))


True

In [30]:
train_collab_with = torch.load(path.join(data_folder, train_collab_with_filename))
train_edges_set = set(map(tuple, train_collab_with.t().tolist()))

/tmp/ipykernel_89638/3191979287.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_collab_with = torch.load(path.join(data_folder, train_collab_with_filename))


In [32]:
train_collab_with

tensor([[   20996,    23967,    32014,  ..., 22734583, 22731172, 22762215],
        [   23967,    20996,    38367,  ..., 22727634, 22762215, 22731172]])

In [27]:
data["artist"].x.shape

torch.Size([1489250, 16])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cuda'


In [6]:
compt_tree_size = [25, 20]

print("Creating test_loader...")
test_loader = LinkNeighborLoader(
    data=data,
    num_neighbors=compt_tree_size,
    neg_sampling_ratio=1,
    edge_label_index=("artist", "collab_with", "artist"),
    batch_size=128,
    num_workers=10,
    pin_memory=True,
)

Creating test_loader...


/home/aleferu/miniforge3/envs/musicbrainz/lib/python3.12/site-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [7]:
class GNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels):
        super().__init__()
        self.metadata = metadata
        self.out_channels = out_channels

        self.conv1 = HeteroConv({
            ("artist", "collab_with", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "has_tag_artists", "tag"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "last_fm_match", "artist"): GATConv((-1, -1), hidden_channels),
            ("track", "has_tag_tracks", "tag"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "linked_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "musically_related_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "personally_related_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("tag", "tags_artists", "artist"): SAGEConv((-1, -1), hidden_channels),
            ("tag", "tags_tracks", "track"): SAGEConv((-1, -1), hidden_channels),
            ("track", "worked_by", "artist"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "worked_in", "track"): SAGEConv((-1, -1), hidden_channels),
        }, aggr="mean")

        self.conv2 = HeteroConv({
            ("artist", "collab_with", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "has_tag_artists", "tag"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "last_fm_match", "artist"): GATConv((-1, -1), hidden_channels),
            ("track", "has_tag_tracks", "tag"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "linked_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "musically_related_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("artist", "personally_related_to", "artist"): GATConv((-1, -1), hidden_channels),
            ("tag", "tags_artists", "artist"): SAGEConv((-1, -1), hidden_channels),
            ("tag", "tags_tracks", "track"): SAGEConv((-1, -1), hidden_channels),
            ("track", "worked_by", "artist"): SAGEConv((-1, -1), hidden_channels),
            ("artist", "worked_in", "track"): SAGEConv((-1, -1), hidden_channels),
        }, aggr="mean")

        self.linear1 = Linear(hidden_channels * 2, hidden_channels * 4)
        self.linear2 = Linear(hidden_channels * 4, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict1 = self.conv1(x_dict, edge_index_dict)
        x_dict2 = self.conv2(x_dict1, edge_index_dict)

        x_artist = torch.cat([x_dict1['artist'], x_dict2['artist']], dim=-1)

        x_artist = self.linear1(x_artist)
        x_artist = self.linear2(x_artist)

        # Normalize the artist node features
        x_artist = F.normalize(x_artist, p=2, dim=-1)

        # Update the dictionary with the new 'artist' features, leaving other nodes unchanged
        x_dict['artist'] = x_artist

        return x_dict

In [8]:
model = GNN(metadata=data.metadata(), hidden_channels=64, out_channels=64).to(device)
model.load_state_dict(
    torch.load("model_2020_3.pth")
)
model.eval()

/tmp/ipykernel_89638/1269635286.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load("model_2020_3.pth")


GNN(
  (conv1): HeteroConv(num_relations=11)
  (conv2): HeteroConv(num_relations=11)
  (linear1): Linear(128, 256, bias=True)
  (linear2): Linear(256, 64, bias=True)
)

In [23]:
def test_model(model, test_loader, device, threshold, train_edges_set):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():  # Disable gradient computation
        for sampled_data in tqdm.tqdm(test_loader):
            # Move data to the device
            sampled_data = sampled_data.to(device)

            # Forward pass
            pred_dict = model(sampled_data.x_dict, sampled_data.edge_index_dict)

            # Get predictions and labels for the 'collab_with' edge type
            edge_label_index = sampled_data['artist', 'collab_with', 'artist'].edge_label_index
            edge_label = sampled_data['artist', 'collab_with', 'artist'].edge_label

            # Convert edge list to set and filter
            test_edges = edge_label_index.t().tolist()
            filtered_edges = [edge for edge in test_edges if tuple(edge) not in train_edges_set]
            print(test_edges)
            print(filtered_edges)
            print(test_edges == filtered_edges)
            break
            if not filtered_edges:
                continue  # Skip if no valid edges left

            filtered_edges = torch.tensor(filtered_edges, dtype=torch.long).t().to(device)
            filtered_labels = edge_label[[i for i, edge in enumerate(test_edges) if tuple(edge) not in train_edges_set]]


            src_emb = pred_dict['artist'][filtered_edges[0]]  # Source node embeddings
            dst_emb = pred_dict['artist'][filtered_edges[1]]  # Destination node embeddings

            # Compute the dot product between source and destination embeddings
            preds = (src_emb * dst_emb).sum(dim=-1)  # Scalar for each edge
            probs = torch.sigmoid(preds)  # Convert logits to probabilities
            preds_binary = (probs > threshold).long()  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_preds.append(preds_binary.cpu())
            all_labels.append(filtered_labels.cpu())
            all_probs.append(probs.cpu())

    # Concatenate all predictions and labels
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_probs = torch.cat(all_probs)

    # Compute metrics
    cm = confusion_matrix(all_labels, all_preds)
    tp = cm[1, 1]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tn = cm[0, 0]
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    roc_auc = roc_auc_score(all_labels, all_probs)

    print("Test Results:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    print(f"Confusion Matrix:\n{tp} {fn}\n{fp} {tn}")

In [24]:
test_model(
    model, test_loader, device, best_threshold, train_edges_set
)

  0%|          | 0/19243 [00:00<?, ?it/s]

[[10, 12], [12, 10], [5, 15], [15, 5], [18, 21], [21, 18], [23, 32], [32, 23], [4, 33], [33, 4], [8, 40], [40, 8], [11, 45], [45, 11], [7, 47], [47, 7], [44, 54], [54, 44], [48, 55], [55, 48], [34, 57], [57, 34], [9, 59], [59, 9], [17, 61], [61, 17], [41, 62], [62, 41], [35, 63], [63, 35], [24, 65], [65, 24], [19, 68], [68, 19], [3, 68], [68, 3], [50, 71], [71, 50], [0, 72], [72, 0], [65, 73], [73, 65], [24, 73], [73, 24], [38, 76], [76, 38], [56, 77], [77, 56], [71, 78], [78, 71], [69, 79], [79, 69], [36, 81], [81, 36], [28, 82], [82, 28], [25, 83], [83, 25], [74, 83], [83, 74], [49, 84], [84, 49], [53, 86], [86, 53], [37, 87], [87, 37], [39, 89], [89, 39], [80, 90], [90, 80], [27, 91], [91, 27], [20, 92], [92, 20], [6, 93], [93, 6], [22, 94], [94, 22], [10, 95], [95, 10], [52, 96], [96, 52], [2, 98], [98, 2], [13, 99], [99, 13], [85, 100], [100, 85], [67, 101], [101, 67], [88, 103], [103, 88], [88, 104], [104, 88], [1, 105], [105, 1], [97, 108], [108, 97], [109, 110], [110, 109], [26

  0%|          | 0/19243 [00:03<?, ?it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [22]:
train_edges_set

{(2280864, 10744334),
 (19997701, 4862128),
 (218679, 6226802),
 (6558110, 15268050),
 (6436874, 17429070),
 (17837245, 1121444),
 (21344932, 12701822),
 (441280, 3146120),
 (9641527, 2196771),
 (9907365, 2762813),
 (5335786, 5849555),
 (18884591, 13909308),
 (11247270, 4539010),
 (16410590, 8592898),
 (822016, 20636334),
 (21745136, 7289020),
 (16363013, 16748219),
 (13790818, 1457115),
 (19227634, 18527743),
 (18039494, 9867052),
 (12385242, 6409015),
 (14760217, 10518667),
 (21546854, 22699637),
 (599547, 16847291),
 (3324080, 8125395),
 (20072958, 15543972),
 (22259519, 20115755),
 (1779658, 4005039),
 (12100784, 10645357),
 (1687952, 15095801),
 (4675906, 195793),
 (7764609, 10789826),
 (11451692, 14382342),
 (13379358, 19477961),
 (19046874, 3756964),
 (21637965, 14338328),
 (16205616, 14049324),
 (5221715, 5854364),
 (378247, 780299),
 (4085945, 15901944),
 (14102680, 2045652),
 (6714099, 5257911),
 (9851775, 8226172),
 (16128761, 22413658),
 (515586, 17960911),
 (22598836, 2263